##### First let's display the mapping we'll be using in a table, and then clean the table up to be used as a mapper for calculation :

In [21]:
import pandas as pd

def convert_md_to_dataframe(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    lines = [line.strip() for line in lines if not line.startswith('|---') and line.strip()]
    
    data = []
    for line in lines[1:]:  # Skip the header line
        # Split by '|' and strip whitespace
        row = [item.strip() for item in line.split('|')]
        # Remove the first and last elements (fillers)
        row = row[1:-1]  # Keep only the relevant columns
        data.append(row)

    # Define the columns
    columns = [
        "Feature Name", "Category", "Sub-Category", "Transaction Tag Key", 
        "Transaction Tag Value", "Transaction Amount value", "Measure", 
        "S2 Source", "Notes", "Data Type"
    ]

    df = pd.DataFrame(data, columns=columns)
    return df

file_path = 'table.md' 
df = convert_md_to_dataframe(file_path)

df

,Feature Name,Category,Sub-Category,Transaction Tag Key,Transaction Tag Value,Transaction Amount value,Measure,S2 Source,Notes,Data Type
0,CATEGORY_ALL_OTHER_CREDITS_CT_AMNT,Proviso,Transaction Tags summary,category,All Other Credits,> 0,sum(txn.Amount)/3 or zero,Proviso banks \ bankAccounts \ transactions \ ...,,currency
1,CATEGORY_ATM_DT_AMNT,Proviso,Transaction Tags summary,category,ATM,< 0,sum(txn.Amount)/3 or zero,Proviso banks \ bankAccounts \ transactions \ ...,,currency
2,CATEGORY_ATM_DT_CNT,Proviso,Transaction Tags summary,category,ATM,< 0,count(txn.Amount)/3 or zero,Proviso banks \ bankAccounts \ transactions \ ...,,number
3,CATEGORY_CREDIT_CARD_REPAYMENTS_DT_CNT,Proviso,Transaction Tags summary,category,Credit Card Repayments,< 0,count(txn.Amount)/3 or zero,Proviso banks \ bankAccounts \ transactions \ ...,,number
4,CATEGORY_DISHONOURS_DT_AMNT,Proviso,Transaction Tags summary,category,Dishonours,< 0,sum(txn.Amount)/3 or zero,Proviso banks \ bankAccounts \ transactions \ ...,,currency
5,CATEGORY_DISHONOURS_DT_CNT,Proviso,Transaction Tags summary,category,Dishonours,< 0,count(txn.Amount)/3 or zero,Proviso banks \ bankAccounts \ transactions \ ...,,number
6,CATEGORY_DONATIONS_BAL_AMNT,Proviso,Transaction Tags summary,category,Donations,,count(txn.Amount)/3 or zero,Proviso banks \ bankAccounts \ transactions \ ...,,currency
7,CATEGORY_EDUCATION_DT_AMNT,Proviso,Transaction Tags summary,category,Education,< 0,sum(txn.Amount)/3 or zero,Proviso banks \ bankAccounts \ transactions \ ...,,currency
8,CATEGORY_FEES_DT_AMNT,Proviso,Transaction Tags summary,category,Fees,< 0,sum(txn.Amount)/3 or zero,Proviso banks \ bankAccounts \ transactions \ ...,,currency
9,CATEGORY_FEES_DT_CNT,Proviso,Transaction Tags summary,category,Fees,< 0,count(txn.Amount)/3 or zero,Proviso banks \ bankAccounts \ transactions \ ...,,number


In [ ]:
%pip install 

In [6]:
# notes-stellare/bank-transform/transform.py

import json
import pandas as pd
import re

def create_mapper_from_table(md_path):
    """
    Create a calculation mapper from a markdown table.
    Args:
        md_path (str): Path to the markdown table file.
    Returns:
        list of dict: Mapper containing rules for feature calculation.
    """
    df = pd.read_csv(md_path, sep='|', engine='python', skipinitialspace=True)
    
    # Drop the first and last rows as they are separators
    df = df.drop(index=[0, len(df)-1], errors='ignore')
    
    # Strip whitespace from headers and rename them appropriately
    df.columns = [col.strip() for col in df.columns]
    
    # Replace NaN with empty strings to avoid type errors
    df = df.fillna('')
    
    mapper = []
    for _, row in df.iterrows():
        # Ensure 'Measure' is a string
        measure = row['Measure'].strip().lower()
        
        # Determine operation and divisor
        operation = 'sum' if 'sum' in measure else 'count'
        divisor = 3 if '/3' in measure else 1
        
        # Handle 'Transaction Amount value'
        amount_condition = row['Transaction Amount value'].strip()
        condition = ''
        if amount_condition:
            # Extract the operator and value (e.g., "> 0" -> ">", "0")
            match = re.match(r'(>=|<=|>|<|==|!=)\s*(\d+\.?\d*)', amount_condition)
            if match:
                operator, value = match.groups()
                condition = f"{operator}{value}"
        
        # Handle 'Transaction Tag Value'; support multiple values separated by commas
        tag_value = row['Transaction Tag Value'].strip().lower()
        tag_values = [val.strip() for val in tag_value.split(',') if val.strip()]
        
        # Check if tag_values contain patterns like "string contains" or "string ilike"
        # If so, extract the actual patterns
        patterns = []
        if any(keyword in tag_values for keyword in ['string contains', 'string ilike']):
            # Extract the actual strings within quotes
            for val in tag_values:
                patterns_found = re.findall(r"'(.*?)'", val)
                patterns.extend(patterns_found)
            tag_values = patterns
            is_regex = True
        else:
            is_regex = False
        
        mapper.append({
            'feature': row['Feature Name'].strip(),
            'category': row['Category'].strip(),
            'tag_key': row['Transaction Tag Key'].strip().lower(),
            'tag_values': tag_values,
            'is_regex': is_regex,
            'data_type': row['Data Type'].strip(),
            'operation': operation,
            'divisor': divisor,
            'condition': condition
        })
    
    return mapper

def calculate_features(mapper, json_path):
    """
    Calculate feature values based on the mapper and JSON data.
    
    Args:
        mapper (list of dict): Mapper containing rules for feature calculation.
        json_path (str): Path to the JSON file.
        
    Returns:
        pd.DataFrame: DataFrame with 'Feature' and 'Value' columns.
    """
    # Load JSON data
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Initialize results dictionary
    results = {rule['feature']: 0 for rule in mapper}
    
    # Process all transactions
    for bank in data.get('data', {}).get('banks', []):
        for account in bank.get('bankAccounts', []):
            for txn in account.get('transactions', []):
                amount = txn.get('amount', 0)
                # Extract tags into a dictionary
                tags = {k.lower(): v.lower() for tag in txn.get('tags', []) for k, v in tag.items()}
                
                for rule in mapper:
                    print("Rule: ", rule)
                    tag_key = rule['tag_key']
                    tag_values = rule['tag_values']
                    is_regex = rule['is_regex']
                    
                    if tag_key in tags:
                        txn_tag_value = tags[tag_key]
                        if is_regex:
                            # Check if any regex pattern matches
                            match = any(re.search(pattern, txn_tag_value) for pattern in tag_values)
                        else:
                            # Check if any of the tag_values are in txn_tag_value
                            match = any(val in txn_tag_value for val in tag_values)
                        
                        if match:
                            condition = rule['condition']
                            condition_met = True
                            if condition:
                                # Safely evaluate the condition
                                try:
                                    condition_met = eval(f"{amount} {condition}")
                                except Exception as e:
                                    print(f"Error evaluating condition '{condition}' for feature '{rule['feature']}': {e}")
                                    condition_met = False
                            
                            if condition_met:
                                if rule['operation'] == 'sum':
                                    results[rule['feature']] += abs(amount) / rule['divisor']
                                elif rule['operation'] == 'count':
                                    results[rule['feature']] += 1 / rule['divisor']
    
    # Create DataFrame with proper formatting
    df = pd.DataFrame(list(results.items()), columns=['Feature', 'Value'])
    
    # Merge with data types
    mapper_df = pd.DataFrame(mapper)
    df = df.merge(mapper_df[['feature', 'data_type']], left_on='Feature', right_on='feature', how='left')
    df = df.drop(columns=['feature'])
    
    # Format values based on data type
    def format_value(row):
        if row['data_type'] == 'currency':
            return f"${row['Value']:,.2f}"
        elif row['data_type'] == 'number':
            return int(row['Value'])
        elif row['data_type'] == 'varchar':
            return row['Value']
        else:
            return row['Value']
    
    df['Value'] = df.apply(format_value, axis=1)
    
    return df[['Feature', 'Value']]

# Usage Example
if __name__ == "__main__":
    mapper = create_mapper_from_table('table.md')
    results_df = calculate_features(mapper, 'nz-bank.json')
    print(results_df)

                                              Feature      Value
0                  CATEGORY_ALL_OTHER_CREDITS_CT_AMNT  $6,576.45
1                                CATEGORY_ATM_DT_AMNT  $3,633.33
2                                 CATEGORY_ATM_DT_CNT         36
3              CATEGORY_CREDIT_CARD_REPAYMENTS_DT_CNT          0
4                         CATEGORY_DISHONOURS_DT_AMNT      $0.00
5                          CATEGORY_DISHONOURS_DT_CNT          0
6                         CATEGORY_DONATIONS_BAL_AMNT      $0.00
7                          CATEGORY_EDUCATION_DT_AMNT      $0.00
8                               CATEGORY_FEES_DT_AMNT      $2.60
9                                CATEGORY_FEES_DT_CNT          1
10                          CATEGORY_GAMBLING_DT_AMNT      $0.00
11                           CATEGORY_GAMBLING_DT_CNT          0
12                            CATEGORY_HEALTH_DT_AMNT     $40.75
13                             CATEGORY_HEALTH_DT_CNT          1
14                   CATE